In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time


base_url = "https://www.jeuxvideo.com/jeux/ps5/jeu-1776970/avis/?p={page_number}"


driver = webdriver.Chrome()


reviews = []
notes = []
dates = []
accords_avis = []
dessaccords_avis = []


for i in range(1, 10):
    url = base_url.format(page_number=i)
    driver.get(url)
    time.sleep(5)  

    
    if i == 1:
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.jad_cmp_paywall_button-cookies.didomi-dismiss-button'))
            )
            cookie_button = driver.find_element(By.CSS_SELECTOR, '.jad_cmp_paywall_button-cookies.didomi-dismiss-button')
            cookie_button.click()
            time.sleep(3)
        except Exception as e:
            print("Le bouton d'acceptation des cookies n'a pas été trouvé ou une erreur est survenue:", e)

    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    
    reviews_elements = soup.find_all("div", class_="txt-avis text-enrichi-forum")
    notes_elements = soup.find_all("div", class_="note-avis")
    dates_elements = soup.find_all("div", class_="bloc-date-avis")
    accords_avis_elements = soup.find_all("span", class_="plus-avis")
    dessaccords_avis_elements = soup.find_all("span", class_="moins-avis")

    
    for review in reviews_elements:
        reviews.append(review.text.strip())

    for note in notes_elements:
        strong = note.find("strong")
        if strong:
            notes.append(strong.text.strip())

    for date in dates_elements:
        a_tag = date.find("a", class_="xXx lien-jv")
        if a_tag:
            dates.append(a_tag.text.strip())

    for accord_avis in accords_avis_elements:
        accords_avis.append(accord_avis.text.strip())

    for dessaccord_avis in dessaccords_avis_elements:
        dessaccords_avis.append(dessaccord_avis.text.strip())


driver.quit()


dict_reviews = {
    "review": reviews,
    "notes": notes,
    "dates": dates,
    "accords_avis": accords_avis,
    "dessaccords_avis": dessaccords_avis
}


df = pd.DataFrame(dict_reviews)
print(df)
df.to_csv("reviews_all_pages.csv", index=False, encoding="utf-8")


                                                review notes  \
0    Belle blague je les eu gratuitement résultat m...     0   
1    EA qui comme d'habitude s'engraisse avec les p...     0   
2    Belle blague je les eu gratuitement résultat m...     0   
3    EA qui comme d'habitude s'engraisse avec les p...     0   
4    Ils ont réussi à faire pire que Fifa 23 ! Mili...     0   
..                                                 ...   ...   
148  5 pour les pelouses et c'est tout........ .......     5   
149  Jeux vraiment horrible script sur script passe...     0   
150  Avant de l’acheter, je trouvais la note des jo...     2   
151  Le gameplay est à chieUne défense de mortLes f...     1   
152  Ceux qui mettent des 0 partout et qui viennent...    19   

                              dates accords_avis dessaccords_avis  
0     Posté le 03 oct. 2023 à 11:54           21                0  
1    Posté le 29 sept. 2023 à 19:06           17                0  
2     Posté le 03 oct. 2023

In [21]:
#!/usr/bin/env python
# coding: utf-8

# In[38]:


import pandas as pd
import re

df = pd.read_csv('reviews_all_pages.csv')
df = df.drop_duplicates(subset='review')

def nettoyer_texte_complet(texte):
    texte = str(texte).lower()
    texte = re.sub(r'[^\w\s]', ' ', texte)  # Supprimer la ponctuation
    texte = re.sub(r'\d+', '', texte)  # Supprimer les chiffres
    texte = re.sub(r'\s+', ' ', texte)  # Supprimer les espaces multiples
    texte = texte.strip()
    
    mots = texte.split()
    
    # Filtrer les mots de deux lettres
    mots_filtres = [mot for mot in mots if len(mot) > 2]
    
    texte = ' '.join(mots_filtres)
    return texte

def convertir_date(date_str):
    date_str = date_str.replace('Posté le ', '')
    date_part = date_str.split(' à ')[0]
    date_part = date_part.replace('.', '')
    
    try:
        jour, mois, annee = date_part.split(' ')
        mois_fr = {
            'janv': '01', 'févr': '02', 'mars': '03', 'avr': '04',
            'mai': '05', 'juin': '06', 'juil': '07', 'août': '08',
            'sept': '09', 'oct': '10', 'nov': '11', 'déc': '12'
        }
        
        mois_num = mois_fr[mois.lower()]
        return f"{jour.zfill(2)}/{mois_num}/{annee}"
    except Exception as e:
        return date_str

# Liste des mots à exclure, incluant les mots de deux lettres
stop_words = set(['et', 'le', 'la', 'un', 'une', 'de', 'à', 'les', 'des', 'pour', 'dans', 'ce', 'en', 'on', 'il', 'je', 'ils']) 

df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['review'] = df['review'].apply(nettoyer_texte_complet)
df['dates'] = df['dates'].apply(convertir_date)

# Enregistrer les données nettoyées dans un nouveau fichier
df.to_csv('nettoyer.csv', index=False)
# Afficher les premières lignes pour vérifier
df.head()

,review,notes,dates,accords_avis,dessaccords_avis
0,belle blague gratuitement résultat mon pote de...,0,03/10/2023,21,0
1,qui comme habitude engraisse avec pigeons mais...,0,29/09/2023,17,0
4,ils ont réussi faire pire que fifa milieu défe...,0,29/09/2023,15,0
5,adepte fifa depuis toujours été convaincu par ...,5,29/09/2023,34,3
6,est pas football joueurs glissent partout aucu...,0,30/09/2023,14,0


In [23]:
from transformers import CamembertTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd 

# Load the tokenizer and model
tokenizer = CamembertTokenizer.from_pretrained("astrosbd/french_emotion_camembert")
model = AutoModelForSequenceClassification.from_pretrained("astrosbd/french_emotion_camembert")

# Load the model into a classification pipeline
classification_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Liste des revues (exemple)
df = pd.read_csv('nettoyer_o.csv')


# Traiter chaque revue avec le pipeline
df['classification'] = df['review'].apply(lambda x: classification_pipe(x)[0]['label']) 

print(df[['review', 'classification']])

df.to_csv('resultats_reviews.csv', index=False)


                                                review classification
0    belle blague gratuitement résultat mon pote de...          anger
1    qui comme habitude engraisse avec pigeons mais...            sad
2    ils ont réussi faire pire que fifa milieu défe...            sad
3    adepte fifa depuis toujours été convaincu par ...            sad
4    est pas football joueurs glissent partout aucu...          anger
..                                                 ...            ...
129                                  pelouses est tout        neutral
130  jeux vraiment horrible script sur script passe...            sad
131  avant acheter trouvais note joueurs sévère mai...          anger
132  gameplay est chieune défense mortles femmes pl...            sad
133  ceux qui mettent partout qui viennent dire mer...            joy

[134 rows x 2 columns]
